# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [1]:
!cat publications.tsv

pub_date	title	venue	excerpt	citation	url_slug	paper_url
16/1/23	"The relationship between journalists and platforms in European online content governance: A case study on a ""non-interference principle''"	European Journal of Communication (Preprint)	Amongst the many debates within the sphere of platform governance, one is particularly telling of the political stakes at play: how should journalistic content be treated in relation to content regulation? There is, perhaps, no better case in which this is playing out than the upcoming Digital Services Act proposed by the European Commission. Specifically, on the one hand, news media organisations have been lobbying for a дnon-interference principleе that would prevent platforms from moderating editorial content that has been published by credible news sources, while on the other hand, officials, platforms, and civil society experts have been arguing against such a measure, primarily, due to the risk of creating an exploitable loophole for

## Import pandas

We are using the very handy pandas library for dataframes.

In [2]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [3]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,16/1/23,The relationship between journalists and platf...,European Journal of Communication (Preprint),Amongst the many debates within the sphere of ...,"Papaevangelou, C. (2023). The relationship bet...",media-exemption,https://shs.hal.science/halshs-03933538v1
1,13/1/23,Funding Intermediaries: Google and Facebookеs ...,Digital Journalism,With more and more governments around the worl...,"Papaevangelou, C. (2023). Funding Intermediari...",fund-intermediaries,https://www.tandfonline.com/doi/full/10.1080/2...
2,4/1/23,The role of citizens in platform governance: A...,Global Media and China,This article proposes to start considering the...,"Papaevangelou, C. (2023). The role of citizens...",paper-title-number-3,https://journals.sagepub.com/doi/abs/10.1177/2...
3,6/9/22,The Case of a Facebook Content Moderation Deba...,Journalism and Digital Content in Emerging Med...,This chapter first discusses how Facebookеs co...,"Papaevangelou, C., Smyrnaios, N. (2022). The C...",fb-como-debacle,https://link.springer.com/chapter/10.1007/978-...
4,9/8/22,Regulating dependency: the political stakes of...,Revista Analisi (Preprint),At a time when the news industry struggles to ...,"Papaevangelou, C., & Smyrnaios, N. (2022). Reg...",reg-dependency,https://shs.hal.science/halshs-03747847/
5,16/9/21,Covidcheck: assessing the implementation of EU...,Dublin City University,CovidCheck is the third monitoring report that...,"Eileen, C., Kirsty, P., Feenane, T., Papaevang...",covidcheck,http://doras.dcu.ie/26472/1/20210914_Final-Rep...
6,15/9/21,Exploring Video-game Production's Contingency ...,"AoIR Selected Papers of Internet Research, 2021",This article studies the cultural contingency ...,"Papaevangelou, C., & Roinioti, E. (2021). EXPL...",twitch-pol-econ,https://journals.uic.edu/ojs/index.php/spir/ar...
7,1/9/21,eSports and Digital Arenas,"Digital Games: Philosophical, social and cultu...",NaN,"Papaevangelou, C., & Roinioti, E. (2021). eSpo...",esports-twitch,NaN
8,1/7/21,The existential stakes of platform governance:...,Open Research Europe,This study introduces a comprehensive overview...,"Papaevangelou, C. (2021). ""The existential sta...",plat-gov-lit-review,https://open-research-europe.ec.europa.eu/arti...


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [4]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [5]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [6]:
!ls ../_publications/

21-covidcheck.md           22-reg-dependency.md
21-esports-twitch.md       23-fund-intermediaries.md
21-plat-gov-lit-review.md  23-media-exemption.md
21-twitch-pol-econ.md      23-paper-title-number-3.md
22-fb-como-debacle.md


In [7]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

cat: ../_publications/2009-10-01-paper-title-number-1.md: No such file or directory
